##  Restore Variables

In [ ]:
%store -r

## Deploying to a Model Endpoint

In [ ]:
%%bash -s "$NEPTUNE_ENDPOINT" "$REGION" "$TRAINING_ID" "$ENDPOINT_ID"

awscurl --region $2 --service neptune-db -X POST https://$1:8182/ml/endpoints -H 'Content-Type: application/json' -d '
    {
        "id" : "'${4}'",
        "mlModelTrainingJobId": "'${3}'"
      }'


## Get the Endpoint Name

In [ ]:
%%bash -s "$NEPTUNE_ENDPOINT" "$ENDPOINT_ID" --out endpoint_response

curl -s https://${1}:8182/ml/endpoints/${2}

In [ ]:
import ast
endpoint_name = ast.literal_eval(endpoint_response)['endpoint']['name']

While the model is being deployed, let's get visualize the computed embeddings

## Visualization

During the model training, Neptune ML will work on producing predictions and calculating the node embeddings then save them in the training output location. In the next cell, we use a helper library to download an visualize the node embeddings saved by Neptune ML

#### Define the graph notebook config
This will be used by the helper library to get the right information about the model trained


In [ ]:
import json
neptune_config = {
  "host": NEPTUNE_ENDPOINT,
  "port": 8182,
  "auth_mode": "DEFAULT",
  "load_from_s3_arn": NEPTUNE_LOAD_ROLE,
  "ssl": True,
  "aws_region": REGION,
  "sparql": {
    "path": "sparql"
  }
}
neptune_config_json = json.dumps(neptune_config, indent = 4)

with open('/home/ec2-user/graph_notebook_config.json', 'w') as file:
    file.write(neptune_config_json)

#### Download the generated embeddings and predictions

In [ ]:
import neptune_ml_utils as neptune_ml

transaction_mapping = neptune_ml.get_node_to_idx_mapping(dataprocessing_job_name=PROCESSING_ID,vertex_label="Transaction")
embeddings = neptune_ml.get_embeddings(training_job_name=TRAINING_ID)
predictions = neptune_ml.get_predictions(training_job_name=TRAINING_ID, class_preds=True)

#### Reduce the embeddings dimensions for visulaization

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

pca = PCA(n_components=3)
pca_result = pca.fit_transform(embeddings)

pcaone = pca_result[:,0]
pcatwo = pca_result[:,1] 
pcathree = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

#### Plot the embeddings in 2D graph

In [ ]:
fig = plt.figure(figsize=(16,12))
fig.suptitle("2D representation of node embeddings")

scatter = plt.scatter(pcaone, pcatwo, c=predictions)
plt.legend(*scatter.legend_elements(), title="isFraud", loc="upper right")
plt.grid()

## Invoke the Deployed Endpoint

Since Neptune ML will deploy an endpoint using Amazon SageMaker, you can also invoke the SageMaker endpoint and generate the score for the fraud label

In [ ]:
import json
import boto3
client = boto3.client('runtime.sagemaker')
data = {"vertices": idss, "topk": 1, "property": "isFraud"} 
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=json.dumps(data))
response_body = response['Body'] 
res = json.loads(response_body.read())
results = []
for i in res['output']['nodes']:
    results.append(i['mlResults'][0]['inferredValue'])

#### Get Original labels to Compute Confusion Matrix

In [ ]:
y_test = [i['isFraud'][0] for i in fraud_labels]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, results)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, results)